<a href="https://colab.research.google.com/github/hellobwjung/colab/blob/master/maketfrecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, glob
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [32]:
import glob
import random
import tqdm
import shutil

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

In [4]:
import tensorflow as tf
tf.__version__

'2.10.0'

# copy 1/10 dataset 

In [10]:
%pwd

'/home/bw.jung/code/core'

# glob 1/10 of mit traning data

In [34]:

## find trining dataset
base_dir = '/dataset/MIT/imgs'
src_dir = os.path.join(base_dir, 'images', 'train')
folders = glob.glob(os.path.join(src_dir, '**', '**'))
# print(os.listdir(base_dir))
# print(src_dir)
# print(folders)

files = []
folders.sort()
folders_src = []
for temp in folders:
    folder_num = temp.split('/')[-1]
#     print(temp, folder_num)
    if int(folder_num) % 10 == 1:
        folders_src.append(temp)
#         print(temp)
        files += glob.glob(os.path.join(temp, '**/**.png'), recursive=True)

print(files[-1])
print('len(files)', len(files))

# dirs = os.listdir(src_dir)
# for idx, d in enumerate(dirs):
#     print(idx, d)

# target folders
new_name = 'images_sub10'
target_dir = os.path.join(base_dir, new_name, 'train')
os.makedirs(target_dir, mode=777, exist_ok=True)

# ## model
print(src_dir)
print(target_dir)



/dataset/MIT/imgs/images/train/moire/001/010188.png
len(files) 260000
/dataset/MIT/imgs/images/train
/dataset/MIT/imgs/images_sub10/train


In [35]:
folders_src

['/dataset/MIT/imgs/images/train/hdrvdp/001',
 '/dataset/MIT/imgs/images/train/hdrvdp/011',
 '/dataset/MIT/imgs/images/train/hdrvdp/021',
 '/dataset/MIT/imgs/images/train/hdrvdp/031',
 '/dataset/MIT/imgs/images/train/hdrvdp/041',
 '/dataset/MIT/imgs/images/train/hdrvdp/051',
 '/dataset/MIT/imgs/images/train/hdrvdp/061',
 '/dataset/MIT/imgs/images/train/hdrvdp/071',
 '/dataset/MIT/imgs/images/train/hdrvdp/081',
 '/dataset/MIT/imgs/images/train/hdrvdp/091',
 '/dataset/MIT/imgs/images/train/hdrvdp/101',
 '/dataset/MIT/imgs/images/train/hdrvdp/111',
 '/dataset/MIT/imgs/images/train/hdrvdp/121',
 '/dataset/MIT/imgs/images/train/hdrvdp/131',
 '/dataset/MIT/imgs/images/train/hdrvdp/141',
 '/dataset/MIT/imgs/images/train/hdrvdp/151',
 '/dataset/MIT/imgs/images/train/hdrvdp/161',
 '/dataset/MIT/imgs/images/train/hdrvdp/171',
 '/dataset/MIT/imgs/images/train/hdrvdp/181',
 '/dataset/MIT/imgs/images/train/hdrvdp/191',
 '/dataset/MIT/imgs/images/train/hdrvdp/201',
 '/dataset/MIT/imgs/images/train/h

# copy 1/10 data to new folder

In [31]:
%cd $base_dir
%ls -la

/dataset/MIT/imgs
total 87455932
drwxr-xr-x  4 1005 1005        4096 Oct 31 11:43 ./
drwxrwxrwx 14 1005 1005        4096 Mar 27  2022 ../
drwxr-xr-x  5 1005 1005        4096 Jan 26  2017 images/
drwxr-xr-x  5 root root        4096 Oct 31 11:45 images_sub10/
-rwxrwxrwx  1 1007 1007 89554845825 Dec 14  2019 joined.zip*


In [43]:
from distutils.dir_util import copy_tree

for idx, f in enumerate(folders_src):
    src = '/'.join(f.split('/')[4:])
    dst = 'images_sub10/' + '/'.join(f.split('/')[5:])
    copy_tree(src, dst)
    print(idx, '] ', src, '-->', dst)
    
#     break
    

0 ]  images/train/hdrvdp/001 --> images_sub10/train/hdrvdp/001
1 ]  images/train/hdrvdp/011 --> images_sub10/train/hdrvdp/011
2 ]  images/train/hdrvdp/021 --> images_sub10/train/hdrvdp/021
3 ]  images/train/hdrvdp/031 --> images_sub10/train/hdrvdp/031
4 ]  images/train/hdrvdp/041 --> images_sub10/train/hdrvdp/041
5 ]  images/train/hdrvdp/051 --> images_sub10/train/hdrvdp/051
6 ]  images/train/hdrvdp/061 --> images_sub10/train/hdrvdp/061
7 ]  images/train/hdrvdp/071 --> images_sub10/train/hdrvdp/071
8 ]  images/train/hdrvdp/081 --> images_sub10/train/hdrvdp/081
9 ]  images/train/hdrvdp/091 --> images_sub10/train/hdrvdp/091
10 ]  images/train/hdrvdp/101 --> images_sub10/train/hdrvdp/101
11 ]  images/train/hdrvdp/111 --> images_sub10/train/hdrvdp/111
12 ]  images/train/hdrvdp/121 --> images_sub10/train/hdrvdp/121
13 ]  images/train/hdrvdp/131 --> images_sub10/train/hdrvdp/131
14 ]  images/train/hdrvdp/141 --> images_sub10/train/hdrvdp/141
15 ]  images/train/hdrvdp/151 --> images_sub10/tra

# check if training files are same

In [46]:
files_dst = glob.glob(os.path.join('/dataset/MIT/imgs/images_sub10/train', '**/**.png'), recursive=True)
len(files_dst)

260000

# Make tfrecords

In [47]:
base_dir = "/dataset/MIT/imgs/images_sub10"

files_train = glob.glob(os.path.join(base_dir, 'train', '**/**.png'), recursive=True)
files_valid  = glob.glob(os.path.join(base_dir, 'test', '**/**.png'), recursive=True)
files_valid += glob.glob(os.path.join(base_dir,  'val', '**/**.png'), recursive=True)
print(len(files_train))
print(len(files_valid))

260000
6000


## shuffle files

In [52]:
random.shuffle(files_train)
random.shuffle(files_valid)

## tfrecords helper function

In [57]:
def write_tfrecords(file_list, idx_split, dataset_name, output_path):

    ext = file_list[0].split('.')[-1]

    tfrecords_name = '%s_%03d.tfrecords' % (dataset_name, idx_split)
    tfrecords_name = os.path.join(output_path, tfrecords_name)

    if os.path.isfile(tfrecords_name):
        print('OHNO -->. tfrecord (%s) exists' % tfrecords_name)
        #return

    print('tfrecord not exists')
    print('--->', tfrecords_name)

    writer = tf.io.TFRecordWriter(tfrecords_name)

    for index, file_name in enumerate(tqdm.tqdm(file_list)):
#         print('%d / %d' % (index+1, len(file_list)))
        
    
        # for png        
        if ext == 'npy':            
            image_arr = np.load(file_name)
        else: 
            image = Image.open(file_name)
            image_arr = np.array(image)
        
        image_byte = image_arr.tobytes()

        example = tf.train.Example(
                features=tf.train.Features(feature={
                    'image': tf.train.Feature(
                            bytes_list=tf.train.BytesList(value=[image_byte])
                    )
                })
        )



        writer.write(example.SerializeToString())
    writer.close()




In [58]:
def make_tfrecords(flist, num_splits, dataset_name, output_path='.'):

 

    assert len(flist) > 0, f'no file is found'
    random.shuffle(flist)
    random.shuffle(flist)

    total_len = len(flist)
    split_len = total_len // num_splits

    print(total_len, split_len)
    start_idx = np.array(list(range(num_splits)) )* split_len
    finish_idx= (np.array((list(range(num_splits)) )) + 1) * split_len

    print(start_idx)
    print(finish_idx)
    finish_idx[-1] = total_len

    # exit()
    # print(start_idx)
    # print(finish_idx)

    for idx_split in range(num_splits):
        print(f'... {idx_split+1} / {num_splits}')
        idx_s = start_idx[idx_split]
        idx_f = finish_idx[idx_split]
        write_tfrecords(flist[idx_s:idx_f], idx_split, dataset_name, output_path)

    print('done make_tfrecords')

In [59]:
%pwd

'/dataset/MIT/imgs'

In [61]:
## make tfrecords
output_path = '/dataset/MIT/tfrecords_sub10'

# train files
# num_of_splits = 20
# make_tfrecords(flist=files_train, num_splits=num_of_splits, dataset_name='mit_sub10_train_%d'%(len(files_train)//num_of_splits), output_path=output_path)

num_of_splits = 1
make_tfrecords(flist=files_valid, num_splits=num_of_splits, dataset_name='mit_sub10_valid_test_%d'%(len(files_train)), output_path=output_path)


6000 6000
[0]
[6000]
... 1 / 1
OHNO -->. tfrecord (/dataset/MIT/tfrecords_sub10/mit_sub10_valid_test_260000_000.tfrecords) exists
tfrecord not exists
---> /dataset/MIT/tfrecords_sub10/mit_sub10_valid_test_260000_000.tfrecords


100%|██████████| 6000/6000 [00:04<00:00, 1311.89it/s]


done make_tfrecords


In [ ]:
%pwd

'/data03/team01/mit'

In [ ]:
# !rm tfrecords/*

# val/teat images

In [ ]:
mydir = 'images'
mydir = 'images_unetv2_mit_unp'
# mydir = 'images_cycle_gan_bayer/'

src_dir = os.path.join(base_dir, mydir, 'val')
folders = glob.glob(os.path.join(src_dir, '**', '**'))
files = []
folders.sort()
for temp in folders:
    files += glob.glob(os.path.join(temp, '**/**.npy'), recursive=True)     
print(files[-1])
print('len(files)', len(files))


src_dir = os.path.join(base_dir, mydir, 'test')
folders = glob.glob(os.path.join(src_dir, '**', '**'))
folders.sort()
for temp in folders:
    files += glob.glob(os.path.join(temp, '**/**.npy'), recursive=True)     
print(files[-1])
print('len(files)', len(files))

 

/data03/team01/mit/images_unetv2_mit_unp/val/moire/000/001761.npy
len(files) 4000
/data03/team01/mit/images_unetv2_mit_unp/test/moire/000/000895.npy
len(files) 6000


In [ ]:
## make tfrecords
num_of_splits = 1
make_tfrecords(flist=files, num_splits=num_of_splits, dataset_name='mit_cyclegan_test_valid', output_path='tfrecords')

6000 6000
[0]
[6000]
... 1 / 1
OHNO -->. tfrecord (tfrecords/mit_cyclegan_test_valid_000.tfrecords) exists
tfrecord not exists
---> tfrecords/mit_cyclegan_test_valid_000.tfrecords


100%|██████████| 6000/6000 [00:04<00:00, 1334.70it/s]


done make_tfrecords


# viz

In [ ]:

mydir = 'images'
mydir = 'images_cycle_gan_bayer'
mydir = 'images_unetv2_mit_unp'
mydir = 'images_unetv2_mit_unp'
mydir = 'images_cycle_gan_bayer'

files = [] 
src_dir = os.path.join(base_dir, mydir, 'test')
folders = glob.glob(os.path.join(src_dir, '**', '**'))
folders.sort()

for temp in folders:
    files += glob.glob(os.path.join(temp, '**/**.npy'), recursive=True)     
files = files[:8]
print(files[-1])
print('len(files)', len(files))

 ## make tfrecords
num_of_splits = 1
make_tfrecords(flist=files, num_splits=num_of_splits, dataset_name='mit_cyclegan_old_viz', output_path='tfrecords')

/data03/team01/mit/images_cycle_gan_bayer/test/hdrvdp/000/000509.npy
len(files) 8
8 8
[0]
[8]
... 1 / 1
tfrecord not exists
---> tfrecords/mit_cyclegan_old_viz_000.tfrecords


100%|██████████| 8/8 [00:00<00:00, 749.90it/s]

done make_tfrecords


In [ ]:
16384/128

128.0

In [ ]:
49152 / 16384

3.0